<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Text-Generation/blob/main/NLP_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import spacy

In [3]:
def read_file(path):
  with open(path) as f:
    my_str = f.read()
  return my_str

In [4]:
!wget https://raw.githubusercontent.com/mr-alamdari/NLP-Text-Generation/main/moby_dick_four_chapters.txt

--2022-05-01 16:45:38--  https://raw.githubusercontent.com/mr-alamdari/NLP-Text-Generation/main/moby_dick_four_chapters.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62685 (61K) [text/plain]
Saving to: ‘moby_dick_four_chapters.txt’

moby_dick_four_chap 100%[===================>]  61.22K  --.-KB/s    in 0.01s   

2022-05-01 16:45:38 (5.27 MB/s) - ‘moby_dick_four_chapters.txt’ saved [62685/62685]



In [5]:
mobydic = read_file('moby_dick_four_chapters.txt')

In [6]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [7]:
nlp.max_length = 1198623

In [8]:
seperate_punc = lambda doc: [token.text.lower() for token in nlp(doc) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [9]:
tokens = seperate_punc(mobydic)

In [10]:
tokens[:20]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and']

In [11]:
len(tokens)

11338

In [12]:
train_len = 25 + 1 # 25 training words , then one target word

text_sequences = [tokens[i-train_len: i] for i in range(train_len, len(tokens))]

In [13]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [14]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [15]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [16]:
' '.join(text_sequences[3])

'some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i thought'

In [17]:
import tensorflow as tf

In [18]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [19]:
tokenizer.fit_on_texts(text_sequences)

In [20]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [21]:
sequences = np.array(sequences)

In [22]:
print(sequences[0])

[ 956   14  263   51  261  408   87  219  129  111  954  260   50   43
   38  315    7   23  546    3  150  259    6 2712   14   24]


In [23]:
print(sequences[1])

[  14  263   51  261  408   87  219  129  111  954  260   50   43   38
  315    7   23  546    3  150  259    6 2712   14   24  957]


In [24]:
print(sequences[2])

[ 263   51  261  408   87  219  129  111  954  260   50   43   38  315
    7   23  546    3  150  259    6 2712   14   24  957    5]


In [25]:
print(sequences[3])

[  51  261  408   87  219  129  111  954  260   50   43   38  315    7
   23  546    3  150  259    6 2712   14   24  957    5   60]


In [26]:
for i in sequences[0]:
  print(i, tokenizer.index_word[i])

956 call
14 me
263 ishmael
51 some
261 years
408 ago
87 never
219 mind
129 how
111 long
954 precisely
260 having
50 little
43 or
38 no
315 money
7 in
23 my
546 purse
3 and
150 nothing
259 particular
6 to
2712 interest
14 me
24 on


In [27]:
for i, count in enumerate(sorted(tokenizer.word_counts.items(), key=lambda x: -x[1])):
  if i == 20:
    break
  print(i, count)

0 ('the', 15540)
1 ('a', 10377)
2 ('and', 9646)
3 ('of', 8287)
4 ('i', 7150)
5 ('to', 6497)
6 ('in', 5647)
7 ('it', 4238)
8 ('that', 3770)
9 ('he', 3247)
10 ('his', 3139)
11 ('was', 2886)
12 ('but', 2652)
13 ('me', 2471)
14 ('with', 2392)
15 ('as', 2366)
16 ('at', 2184)
17 ('this', 2158)
18 ('you', 2158)
19 ('is', 1950)


In [28]:
vocab_size=len(tokenizer.word_counts)

In [29]:
vocab_size

2717

In [32]:
x = sequences[:, :-1]

In [33]:
y = sequences[:, -1]

In [34]:
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size+1)

In [35]:
seq_len = x.shape[1]

In [45]:
def create_model(vocab_size, seq_len):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, seq_len, input_length=seq_len))
  model.add(tf.keras.layers.LSTM(seq_len, return_sequences=True))
  model.add(tf.keras.layers.LSTM(seq_len))
  model.add(tf.keras.layers.Dense(seq_len, activation='relu'))
  model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

  model.summary()
  return model

In [48]:
model = create_model(vocab_size+1, seq_len)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 25, 25)            67950     
                                                                 
 lstm_4 (LSTM)               (None, 25, 25)            5100      
                                                                 
 lstm_5 (LSTM)               (None, 25)                5100      
                                                                 
 dense_4 (Dense)             (None, 25)                650       
                                                                 
 dense_5 (Dense)             (None, 2718)              70668     
                                                                 
Total params: 149,468
Trainable params: 149,468
Non-trainable params: 0
_________________________________________________________________


In [49]:
from pickle import dump, load

In [70]:
model.fit(x, y, batch_size=32, epochs=20, verbose=2)

Epoch 1/20
354/354 - 9s - loss: 5.9622 - accuracy: 0.0586 - 9s/epoch - 26ms/step
Epoch 2/20
354/354 - 9s - loss: 5.8368 - accuracy: 0.0643 - 9s/epoch - 26ms/step
Epoch 3/20
354/354 - 9s - loss: 5.7486 - accuracy: 0.0657 - 9s/epoch - 26ms/step
Epoch 4/20
354/354 - 9s - loss: 5.6548 - accuracy: 0.0675 - 9s/epoch - 26ms/step
Epoch 5/20
354/354 - 9s - loss: 5.5733 - accuracy: 0.0690 - 9s/epoch - 26ms/step
Epoch 6/20
354/354 - 9s - loss: 5.5074 - accuracy: 0.0738 - 9s/epoch - 25ms/step
Epoch 7/20
354/354 - 9s - loss: 5.4458 - accuracy: 0.0758 - 9s/epoch - 26ms/step
Epoch 8/20
354/354 - 9s - loss: 5.3802 - accuracy: 0.0785 - 9s/epoch - 26ms/step
Epoch 9/20
354/354 - 9s - loss: 5.3125 - accuracy: 0.0823 - 9s/epoch - 26ms/step
Epoch 10/20
354/354 - 9s - loss: 5.2486 - accuracy: 0.0828 - 9s/epoch - 26ms/step
Epoch 11/20
354/354 - 9s - loss: 5.1911 - accuracy: 0.0852 - 9s/epoch - 26ms/step
Epoch 12/20
354/354 - 9s - loss: 5.1332 - accuracy: 0.0887 - 9s/epoch - 27ms/step
Epoch 13/20
354/354 - 9s 

In [76]:
model.save('mobydic_model.h5')

In [77]:
dump(tokenizer, open('simpleTokenizer', 'wb'))

In [73]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
  output_text = []
  input_text = seed_text
  for i in range(num_gen_words):
    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = tf.keras.preprocessing.sequence.pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
    pred_word = model.predict(pad_encoded, verbose=0)[0]
    pred_word_ind = np.argmax(pred_word)
    pred_word = tokenizer.index_word[pred_word_ind]
    input_text += ' '+pred_word
    output_text.append(pred_word)
  return ' '.join(output_text)

In [74]:
num_gen_words = 20
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

'the man and be not be was was be be and be and be not be not be not be'

In [75]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 9s - loss: 4.6543 - accuracy: 0.1146 - 9s/epoch - 26ms/step
Epoch 2/100
354/354 - 9s - loss: 4.5950 - accuracy: 0.1180 - 9s/epoch - 25ms/step
Epoch 3/100
354/354 - 9s - loss: 4.5381 - accuracy: 0.1201 - 9s/epoch - 25ms/step
Epoch 4/100
354/354 - 9s - loss: 4.4843 - accuracy: 0.1255 - 9s/epoch - 26ms/step
Epoch 5/100
354/354 - 9s - loss: 4.4360 - accuracy: 0.1284 - 9s/epoch - 25ms/step
Epoch 6/100
354/354 - 9s - loss: 4.3852 - accuracy: 0.1302 - 9s/epoch - 25ms/step
Epoch 7/100
354/354 - 9s - loss: 4.3371 - accuracy: 0.1339 - 9s/epoch - 25ms/step
Epoch 8/100
354/354 - 9s - loss: 4.2858 - accuracy: 0.1386 - 9s/epoch - 25ms/step
Epoch 9/100
354/354 - 9s - loss: 4.2336 - accuracy: 0.1408 - 9s/epoch - 25ms/step
Epoch 10/100
354/354 - 9s - loss: 4.1891 - accuracy: 0.1475 - 9s/epoch - 25ms/step
Epoch 11/100
354/354 - 9s - loss: 4.1395 - accuracy: 0.1479 - 9s/epoch - 25ms/step
Epoch 12/100
354/354 - 9s - loss: 4.0992 - accuracy: 0.1514 - 9s/epoch - 25ms/step
Epoch 13/100


In [78]:
num_gen_words = 40
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

"n't do n't be not be not almost be been four the original of if it is a southerner and said the landlord and presently to be two off i say not the urbane plaster man i abandon the business"